In [1]:
from static_huff import static_huffman
from adaptive_huff import adaptive_huffman_encode

import static_huff
import adaptive_huff

In [2]:
root = static_huffman("abracadabra")
root

-----HUFFMAN TREE-----
#11
0 -> #5 a
1 -> #6
 0 -> #2
  0 -> #1 c
  1 -> #1 d
 1 -> #4
  0 -> #2 r
  1 -> #2 b

In [3]:
encoded = static_huff.encode("abracadabra")
print(encoded)

bitarray('01111100100010101111100')


In [4]:
decoded = static_huff.decode(encoded, root)
print(decoded)

abracadabra


In [7]:
## FIXME => swapping part!

text = "aardvaa" # here propery is broken!
adaptive_huff.adaptive_huffman_encode(text, N = 9)
pass

-----HUFFMAN TREE-----
#{ W=1, N=9 }
 0 -> #{ W=0, N=7 } => NYT
 1 -> #{ W=1, N=8 } => a
Leaves: {'NYT': #{ W=0, N=7 } => NYT, 'a': #{ W=1, N=8 } => a}
Weight Classes: {0: {#{ W=0, N=7 } => NYT}, 1: {#{ W=1, N=8 } => a, #{ W=1, N=9 }}}
----------------------
-----HUFFMAN TREE-----
#{ W=2, N=9 }
 0 -> #{ W=0, N=7 } => NYT
 1 -> #{ W=2, N=8 } => a
Leaves: {'NYT': #{ W=0, N=7 } => NYT, 'a': #{ W=2, N=8 } => a}
Weight Classes: {0: {#{ W=0, N=7 } => NYT}, 1: set(), 2: {#{ W=2, N=8 } => a, #{ W=2, N=9 }}}
----------------------
-----HUFFMAN TREE-----
#{ W=3, N=9 }
 0 -> #{ W=1, N=7 }
  0 -> #{ W=0, N=5 } => NYT
  1 -> #{ W=1, N=6 } => r
 1 -> #{ W=2, N=8 } => a
Leaves: {'NYT': #{ W=0, N=5 } => NYT, 'a': #{ W=2, N=8 } => a, 'r': #{ W=1, N=6 } => r}
Weight Classes: {0: {#{ W=0, N=5 } => NYT}, 1: {#{ W=1, N=6 } => r, #{ W=1, N=7 }}, 2: {#{ W=2, N=8 } => a}, 3: {#{ W=3, N=9 }}}
----------------------
-----HUFFMAN TREE-----
#{ W=4, N=9 }
 0 -> #{ W=2, N=7 }
  0 -> #{ W=1, N=5 }
   0 -> #{ W=0, N=